# 01 — Raw Data Audit (HCS)
This notebook audits the raw Excel file without modifying it.
Scope:
- Load sheets: `cohorte_and`, `cohorte_rs`
- Basic profiling: schema, missingness, duplicates, value sanity checks
- Export audit artifacts (metadata + missingness summary)

In [32]:
from __future__ import annotations

import os
from pathlib import Path
from datetime import datetime
import hashlib

import numpy as np
import pandas as pd

pd.set_option("display.max_columns", 200)
pd.set_option("display.max_rows", 200)
pd.set_option("display.width", 200)

In [33]:
# --- User config ---
RAW_PATH = Path("../../data/raw_data/raw_data.xlsx")
SHEETS = ["cohorte_and", "cohorte_rs"]

OUT_DIR = Path("../../data/audit")
OUT_DIR.mkdir(parents=True, exist_ok=True)

RUN_TAG = datetime.now().strftime("%Y%m%d_%H%M%S")


In [34]:
def file_sha256(path: Path, chunk_size: int = 1024 * 1024) -> str:
    h = hashlib.sha256()
    with open(path, "rb") as f:
        while True:
            chunk = f.read(chunk_size)
            if not chunk:
                break
            h.update(chunk)
    return h.hexdigest()

assert RAW_PATH.exists(), f"Raw file not found: {RAW_PATH}"

raw_hash = file_sha256(RAW_PATH)
raw_size_mb = RAW_PATH.stat().st_size / (1024**2)

print("RAW_PATH:", RAW_PATH)
print("SHA256:", raw_hash)
print(f"Size: {raw_size_mb:.2f} MB")


RAW_PATH: ../../data/raw_data/raw_data.xlsx
SHA256: 34e75e75253dc896f736050ee627730beb9a9f2f33c0ffcf8bb8f10f25ecc38e
Size: 0.06 MB


In [35]:
xls = pd.ExcelFile(RAW_PATH)
print("Available sheets:")
for s in xls.sheet_names:
    print(" -", s)

missing = [s for s in SHEETS if s not in xls.sheet_names]
assert not missing, f"Missing required sheets: {missing}"

Available sheets:
 - Apos
 - cohorte_and
 - cohorte_rs
 - Hoja1


In [36]:
def read_sheet(sheet_name: str) -> pd.DataFrame:
    df = pd.read_excel(RAW_PATH, sheet_name=sheet_name, dtype_backend="numpy_nullable")
    df.columns = [str(c).strip() for c in df.columns]
    return df

df_and = read_sheet("cohorte_and")
df_rs  = read_sheet("cohorte_rs")

print("cohorte_and shape:", df_and.shape)
print("cohorte_rs  shape:", df_rs.shape)

df_and.head(3)


cohorte_and shape: (83, 16)
cohorte_rs  shape: (77, 19)


,id,edad,peso,talla,imc,glicemia,pas1t,pad1t,ct1,ct3,tg1,tg3,hdl1,hdl3,ldl1,ldl3
0,1,33,68.0,169,23.8,76.2,101,60,239,426,85.1,386,69.0,43,153.0,305
1,2,37,58.8,162,22.4,71.9,101,58,222,399,97.0,356,65.6,50,137.0,278
2,3,30,63.0,160,24.6,79.8,113,58,197,396,100.0,356,56.3,55,120.7,269


## Quick overview
We inspect:
- column names and types
- missingness
- duplicates (rows and candidate IDs)
- basic sanity checks (ranges)

In [37]:
def audit_schema(df: pd.DataFrame, sheet: str) -> pd.DataFrame:
    out = pd.DataFrame({
        "sheet": sheet,
        "column": df.columns,
        "dtype": [str(t) for t in df.dtypes],
        "n_non_null": df.notna().sum().values,
        "n_null": df.isna().sum().values,
        "pct_null": (df.isna().mean().values * 100).round(2),
        "n_unique": [df[c].nunique(dropna=True) for c in df.columns],
    })
    return out.sort_values(["pct_null", "n_unique"], ascending=[False, True]).reset_index(drop=True)

def audit_duplicates(df: pd.DataFrame) -> dict:
    return {
        "n_rows": len(df),
        "n_duplicate_rows": int(df.duplicated().sum()),
        "pct_duplicate_rows": float(df.duplicated().mean() * 100),
    }

def describe_numeric(df: pd.DataFrame, sheet: str) -> pd.DataFrame:
    num_cols = df.select_dtypes(include=["number", "Float64", "Int64"]).columns.tolist()
    if not num_cols:
        return pd.DataFrame()
    desc = df[num_cols].describe(percentiles=[.01, .05, .5, .95, .99]).T
    desc.insert(0, "sheet", sheet)
    desc.insert(1, "column", desc.index)
    return desc.reset_index(drop=True)


In [38]:
schema_and = audit_schema(df_and, "cohorte_and")
schema_rs  = audit_schema(df_rs,  "cohorte_rs")

schema = pd.concat([schema_and, schema_rs], ignore_index=True)

schema.head(20)

,sheet,column,dtype,n_non_null,n_null,pct_null,n_unique
0,cohorte_and,edad,Int64,83,0,0.00,15
1,cohorte_and,talla,Int64,83,0,0.00,24
2,cohorte_and,peso,Float64,83,0,0.00,33
3,cohorte_and,pas1t,Int64,83,0,0.00,35
4,cohorte_and,pad1t,Int64,83,0,0.00,37
5,cohorte_and,hdl3,Int64,83,0,0.00,43
6,cohorte_and,ct1,Int64,83,0,0.00,55
7,cohorte_and,imc,Float64,83,0,0.00,59
8,cohorte_and,tg1,Float64,83,0,0.00,64
9,cohorte_and,ldl3,Int64,83,0,0.00,65


In [39]:
schema_out = OUT_DIR / f"{RUN_TAG}_schema_missingness.csv"
schema.to_csv(schema_out, index=False)
print("Saved:", schema_out)

Saved: ../../data/audit/20260129_091325_schema_missingness.csv


In [40]:
dup_and = audit_duplicates(df_and)
dup_rs  = audit_duplicates(df_rs)

dup_summary = pd.DataFrame([
    {"sheet": "cohorte_and", **dup_and},
    {"sheet": "cohorte_rs", **dup_rs},
])

dup_summary

,sheet,n_rows,n_duplicate_rows,pct_duplicate_rows
0,cohorte_and,83,0,0.0
1,cohorte_rs,77,0,0.0


In [41]:
desc_and = describe_numeric(df_and, "cohorte_and")
desc_rs  = describe_numeric(df_rs,  "cohorte_rs")
desc = pd.concat([desc_and, desc_rs], ignore_index=True)

desc.head(20)


,sheet,column,count,mean,std,min,1%,5%,50%,95%,99%,max
0,cohorte_and,id,83.0,42.0,24.103942,1.0,1.82,5.1,42.0,78.9,82.18,83.0
1,cohorte_and,edad,83.0,32.0,3.327637,24.0,24.0,26.1,32.0,37.0,38.0,38.0
2,cohorte_and,peso,83.0,60.83494,10.044917,42.0,45.28,49.0,59.0,80.9,90.08,95.0
3,cohorte_and,talla,83.0,163.46988,6.091452,150.0,151.64,154.0,163.0,172.0,176.36,178.0
4,cohorte_and,imc,83.0,22.705783,3.013402,17.03,17.8254,18.8,22.0,27.97,30.686,32.9
5,cohorte_and,glicemia,83.0,79.615663,5.844666,64.7,67.98,69.82,79.7,89.79,93.736,98.0
6,cohorte_and,pas1t,83.0,106.951807,10.359282,82.0,87.74,92.2,106.0,125.9,135.9,140.0
7,cohorte_and,pad1t,83.0,63.60241,10.588696,27.0,40.12,48.1,63.0,79.9,89.98,99.0
8,cohorte_and,ct1,83.0,185.120482,26.250388,134.0,138.92,146.1,184.0,227.5,242.6,259.0
9,cohorte_and,ct3,83.0,274.240964,51.215135,169.0,172.28,189.2,278.0,372.1,403.86,426.0


In [42]:
desc_out = OUT_DIR / f"{RUN_TAG}_numeric_describe.csv"
desc.to_csv(desc_out, index=False)
print("Saved:", desc_out)

Saved: ../../data/audit/20260129_091325_numeric_describe.csv


## Sanity checks (ranges)
These checks are **non-destructive**: they only flag suspicious values.

In [43]:
# ---- EDIT THESE to match column names in your sheets ----
RANGE_RULES = {
    "edad":   (12, 55),     
    "imc":    (12, 80),     
    "eg_1tm": (4, 16),      
    "eg_3tm": (24, 44),     
    "ct_1tm": (80, 450),
    "ct_3tm": (80, 600),    
    "hdl_1tm": (10, 150),
    "ldl_1tm": (10, 450),
    "tg_1tm":  (20, 1000),  
    "tg_3tm":  (20, 2000),  
}

def range_violations(df: pd.DataFrame, sheet: str, rules: dict) -> pd.DataFrame:
    rows = []
    for col, (lo, hi) in rules.items():
        if col not in df.columns:
            continue
        s = df[col]
        if not pd.api.types.is_numeric_dtype(s):
            continue
        mask = s.notna() & ((s < lo) | (s > hi))
        n_bad = int(mask.sum())
        if n_bad > 0:
            rows.append({
                "sheet": sheet,
                "column": col,
                "lo": lo,
                "hi": hi,
                "n_violations": n_bad,
                "examples": s[mask].head(10).tolist(),
            })
    return pd.DataFrame(rows)

viol_and = range_violations(df_and, "cohorte_and", RANGE_RULES)
viol_rs  = range_violations(df_rs,  "cohorte_rs",  RANGE_RULES)

viol = pd.concat([viol_and, viol_rs], ignore_index=True)
viol if len(viol) else "No range violations detected (or RANGE_RULES not configured)."


'No range violations detected (or RANGE_RULES not configured).'

In [44]:
viol_out = OUT_DIR / f"{RUN_TAG}_sanity_range_violations.csv"
viol.to_csv(viol_out, index=False)
print("Saved:", viol_out)

Saved: ../../data/audit/20260129_091325_sanity_range_violations.csv


## Audit summary
Artifacts generated:
- schema + missingness
- numeric describe
- range violations
Next notebook (02): cohort definition + inclusion/exclusion + target MSPH definition.


In [45]:
import json

audit_run = {
    "run_tag": RUN_TAG,
    "raw_path": str(RAW_PATH),
    "raw_sha256": raw_hash,
    "raw_size_mb": raw_size_mb,
    "sheets": SHEETS,
    "shapes": {
        "cohorte_and": list(df_and.shape),
        "cohorte_rs": list(df_rs.shape),
    },
    "outputs": {
        "schema_missingness": str(schema_out),
        "numeric_describe": str(desc_out),
        "sanity_violations": str(viol_out),
    }
}

run_out = OUT_DIR / f"{RUN_TAG}_audit_run.json"
run_out.write_text(json.dumps(audit_run, indent=2), encoding="utf-8")
print("Saved:", run_out)


Saved: ../../data/audit/20260129_091325_audit_run.json
